In [1]:
import subprocess

In [6]:
subprocess.run("ls -la /dev/null", shell=True)

In [7]:
subprocess.run("python -c 'print(1/ 7)'", shell=True)

In [8]:
subprocess.run("false", shell=True)

In [9]:
import socket

In [10]:
def grep(pattern):
    print("start grep for", pattern)
    while True:
        s = yield
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))

In [16]:
g = grep("hello")
next(g)

In [17]:
g.send("hello world")

In [18]:
g.send("dwkjdkwjjd world")

In [15]:
g.send("dwdw")

In [19]:
g.throw(ValueError("err"))

In [20]:
g.send("dwdw")

In [21]:
import asyncio

In [22]:
@asyncio.coroutine
def old_coro():
    yield 1
    yield 2



In [31]:
async def countdown(n):
    print(f"start count_{n}")
    for i in range(n):
        print(f"count_{n}: {i}")
        await asyncio.sleep(1)

In [35]:
import time

t1 = time.time()

c = countdown(5)
print(c)

await c

t2 = time.time()
print("time", t2 - t1)

In [36]:
t1 = time.time()

await countdown(5)
await countdown(6)

t2 = time.time()
print("time", t2 - t1)

In [40]:
t1 = time.time()

task1 = asyncio.create_task(countdown(5))
print("created task")

await countdown(6)
print("end count 6")

await task1
print("end task")

t2 = time.time()
print("time", t2 - t1)

In [42]:
t1 = time.time()

tasks = [
    asyncio.create_task(countdown(i))
    for i in (5, 6, 3, 4)
]
print("created tasks")

await asyncio.gather(*tasks)
print("end gather")

t2 = time.time()
print("time", t2 - t1)

In [51]:
async def countdown(n):
    print(f"start count_{n}")
    for i in range(n):
        print(f"count_{n}: {i}")
        await asyncio.sleep(1)
    
    return n

t1 = time.time()

tasks = [
    asyncio.create_task(countdown(i))
    for i in (5, 6, 3, 4)
]
print("created tasks")

for coro in asyncio.as_completed(tasks):
    res = await coro
    print(res, coro)

print("end gather")

t2 = time.time()
print("time", t2 - t1)

In [53]:
async def countdown(n):
    print(f"start count_{n}")
    for i in range(n):
        print(f"count_{n}: {i}")
        await asyncio.sleep(1)
    
    return n


async def block_coro(n):
    time.sleep(n)


t1 = time.time()

tasks = [
    asyncio.create_task(countdown(5)),
    asyncio.create_task(block_coro(5)),
]
print("created tasks")

await asyncio.gather(*tasks)

print("end gather")

t2 = time.time()
print("time", t2 - t1)

In [54]:
asyncio.get_event_loop()

In [56]:
!pip install aiohttp

In [57]:
import aiohttp

In [86]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 50


async def fetch(url, session):
    async with session.get(url) as resp:
        data = await resp.read()
        assert resp.status == 200
        #print(resp.status, len(data))
    

async def batch_fetch(urls):
    async with aiohttp.ClientSession() as session:
        for url in urls:
            await fetch(url, session)

t1 = time.time()

await batch_fetch(URLS)

t2 = time.time()
print("time", t2 - t1)

In [79]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 50


async def fetch(url, session, sem):
    async with sem:
        async with session.get(url) as resp:
            data = await resp.read()
            assert resp.status == 200
            #print(resp.status, len(data))
    

async def batch_fetch(urls, sem):
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch(url, session, sem))
            for url in urls
        ]
        await asyncio.gather(*tasks)

            
t1 = time.time()

sem = asyncio.Semaphore(10)
await batch_fetch(URLS, sem)

t2 = time.time()
print("time", t2 - t1)

In [89]:
URL = "https://docs.python.org/3/whatsnew/3.11.html"
URLS = [URL] * 50
counter = 0


async def fetch(session, q):
    while True:
        url = await q.get()
        global counter
        counter += 1

        try:
            async with session.get(url) as resp:
                data = await resp.read()
                assert resp.status == 200
        finally:
            q.task_done()
    

async def batch_fetch(urls, workers=5):
    q = asyncio.Queue()
    for url in urls:
        await q.put(url)

    async with aiohttp.ClientSession() as session:
        workers = [
            asyncio.create_task(fetch(session, q))
            for _ in range(workers)
        ]
        await q.join()
        
        for w in workers:
            w.cancel()


t1 = time.time()

await batch_fetch(URLS, workers=1)

t2 = time.time()
print("time", t2 - t1)
print("count", counter)

In [90]:
event = asyncio.get_event_loop()